In [39]:
import json
from subprocess import Popen
import shlex
import os
import jsonlines

In [67]:
input_path = "/home/lhk/data/hp_test/hp_test_results.json"

In [68]:
with open(input_path) as file:
    data = json.load(file)

In [85]:
parent_dir, _ = os.path.split(input_path)
parent_dir += "/"
for entry in data:
    in_path = parent_dir+entry["result_filepath"]
    result_path, _ = os.path.split(in_path)
    
    parsed = {in_path:[]}
    for obj in jsonlines.open(in_path):
        parsed[in_path].append({"id":obj['id'], "model_completions": obj["model_completions"]})
        
    prepared_path = result_path + "/cc_eval_format.json"

    with open(prepared_path, "w") as file:
        json.dump(parsed, file)
    
    out_path = result_path + "/output.json"
    
    cmd = f"bazel run -c opt execution:solve_example -- --data_path=/home/lhk/data/ --input_path={prepared_path} --output_path={out_path}   --python3_path=/usr/bin/python3.10 --python3_library_paths=/usr/lib/python3.10"
    
    Popen(shlex.split(cmd))
    
print("done")

Loading: 
Loading: 0 packages loaded
Analyzing: target //execution:solve_example (0 packages loaded, 0 targets configured)
INFO: Analyzed target //execution:solve_example (0 packages loaded, 0 targets configured).
INFO: Found 1 target...
[0 / 1] [Prepa] BazelWorkspaceStatusAction stable-status.txt
Target //execution:solve_example up-to-date:
  bazel-bin/execution/solve_example
INFO: Elapsed time: 0.075s, Critical Path: 0.00s
INFO: 1 process: 1 internal.
INFO: Build completed successfully, 1 total action
INFO: Running command line: bazel-bin/execution/solve_example '--data_path=/home/lhk/data/' '--input_path=/home/lhk/data/hp_test/0/frequency_penalty-0.2-temperature-0/cc_eval_format.json' '--output_path=/home/lhk/data/hp_test/0/frequency_penalty-0.2-temperature-0/output.json' '--python3_path=/usr/bin/python3.10' '--python3_library_paths=/usr/lib/python3.10'
INFO: Build completed successfully, 1 total action
Loading: 
Loading: 0 packages loaded
Analyzing: target //execution:solve_example

/home/lhk/data/hp_test/0/frequency_penalty-0.2-temperature-0/predictions.txt
parsed the input json
found a generation
/home/lhk/data/hp_test/0/frequency_penalty-0-temperature-0.1/predictions.txt
parsed the input json
compilation error
compilation error
failed


  bazel-bin/execution/solve_example
[global_forkclient.cc : 122] RAW: Starting global forkserver
INFO: Elapsed time: 0.080s, Critical Path: 0.00s
INFO: 1 process: 1 internal.
INFO: Build completed successfully, 1 total action
INFO: Running command line: bazel-bin/execution/solve_example '--data_path=/home/lhk/data/' '--input_path=/home/lhk/data/hp_test/0/frequency_penalty-0-temperature-0.1/cc_eval_format.json' '--output_path=/home/lhk/data/hp_test/0/frequency_penalty-0-temperature-0.1/output.json' '--python3_path=/usr/bin/python3.10' '--python3_library_paths=/usr/lib/python3.10'
INFO: Build completed successfully, 1 total action
Loading: 
Loading: 0 packages loaded
Analyzing: target //execution:solve_example (0 packages loaded, 0 targets configured)
INFO: Analyzed target //execution:solve_example (0 packages loaded, 0 targets configured).
INFO: Found 1 target...
[0 / 1] [Prepa] BazelWorkspaceStatusAction stable-status.txt
Target //execution:solve_example up-to-date:
  bazel-bin/executi

found a generation
found a generation
/home/lhk/data/hp_test/0/frequency_penalty-0-temperature-0/predictions.txt
parsed the input json
failed
found a generation
done
failed
found a generation
found a generation
/home/lhk/data/hp_test/0/frequency_penalty-0.2-temperature-0.1/predictions.txt
parsed the input json
failed
found a generation
failed
found a generation
failed
found a generation
failed
found a generation
compilation error


Loading: 
Loading: 0 packages loaded
Analyzing: target //execution:solve_example (0 packages loaded, 0 targets configured)
INFO: Analyzed target //execution:solve_example (0 packages loaded, 0 targets configured).
INFO: Found 1 target...
[0 / 1] [Prepa] BazelWorkspaceStatusAction stable-status.txt
Target //execution:solve_example up-to-date:
  bazel-bin/execution/solve_example
INFO: Elapsed time: 0.091s, Critical Path: 0.00s
[global_forkclient.cc : 122] RAW: Starting global forkserver
INFO: 1 process: 1 internal.
INFO: Build completed successfully, 1 total action
INFO: Running command line: bazel-bin/execution/solve_example '--data_path=/home/lhk/data/' '--input_path=/home/lhk/data/hp_test/0/frequency_penalty-0.2-temperature-0.1/cc_eval_format.json' '--output_path=/home/lhk/data/hp_test/0/frequency_penalty-0.2-temperature-0.1/output.json' '--python3_path=/usr/bin/python3.10' '--python3_library_paths=/usr/lib/python3.10'
INFO: Build completed successfully, 1 total action


compilation error
failed
found a generation
found a generation
failed
failed
found a generation
found a generation
failed
found a generation
failed
failed
found a generation
found a generation
compilation error
compilation error
failed
found a generation


[global_forkclient.cc : 122] RAW: Starting global forkserver


failed
found a generation
failed
found a generation


<Popen: returncode: None args: ['bazel', 'run', '-c', 'opt', 'execution:solv...>

Loading: 
Loading: 0 packages loaded
Analyzing: target //execution:solve_example (0 packages loaded, 0 targets configured)
INFO: Analyzed target //execution:solve_example (1 packages loaded, 3 targets configured).
INFO: Found 1 target...
[0 / 4] [Prepa] BazelWorkspaceStatusAction stable-status.txt
[44 / 708] Compiling src/google/protobuf/compiler/cpp/cpp_message.cc; 1s linux-sandbox ... (12 actions, 11 running)
[49 / 708] Compiling src/google/protobuf/compiler/cpp/cpp_message.cc; 2s linux-sandbox ... (12 actions, 11 running)
[54 / 708] Compiling src/google/protobuf/compiler/cpp/cpp_message.cc; 3s linux-sandbox ... (12 actions, 11 running)
[60 / 708] Compiling src/google/protobuf/compiler/cpp/cpp_message.cc; 4s linux-sandbox ... (12 actions, 11 running)
[65 / 708] Compiling src/google/protobuf/compiler/cpp/cpp_message.cc; 5s linux-sandbox ... (12 actions running)
[72 / 708] Compiling src/google/protobuf/compiler/cpp/cpp_message.cc; 7s linux-sandbox ... (12 actions running)
[77 / 708] Co

/home/lhk/data/hp_test/0/frequency_penalty-0.2-temperature-0.1/predictions.txt
parsed the input json
found a generation
failed
found a generation
passed
found a generation
failed
found a generation
compilation error
compilation error
failed
found a generation
compilation error
compilation error
failed
found a generation
compilation error
compilation error
failed
found a generation


I20221119 22:01:14.642277 27364 monitor.cc:386] Stack traces have been disabled


failed
found a generation


I20221119 22:01:19.702234 27454 monitor.cc:386] Stack traces have been disabled
I20221119 22:01:24.710242 27467 monitor.cc:386] Stack traces have been disabled


failed
found a generation
failed
found a generation
passed
found a generation
compilation error
compilation error
failed
found a generation
compilation error
compilation error
failed
found a generation
compilation error
compilation error
failed
found a generation
compilation error
compilation error
failed
found a generation
failed
found a generation
failed
found a generation
passed
found a generation
compilation error
compilation error
failed
writing output to: /home/lhk/data/hp_test/0/frequency_penalty-0.2-temperature-0.1/output.json
